[**Blueprints for Text Analysis Using Python**](https://github.com/blueprints-for-text-analytics-python/blueprints-text)  
Jens Albrecht, Sidharth Ramachandran, Christian Winkler

# Chapter 3:<div class='tocSkip'/>

# Scraping Websites and Extracting Data

## Remark<div class='tocSkip'/>

The code in this notebook differs slightly from the printed book. The Reuters website has changed significantly and is now obfuscated. For the illustractive examples we use the version from the Internet Archive. There are some utility functions which explain how to handle obfuscation though.

Moreover, several layout and formatting commands, like `figsize` to control figure size or subplot commands are removed in the book.

All of this is done to simplify the code in the book and put the focus on the important parts instead of formatting.

## Setup<div class='tocSkip'/>

Set directory locations. If working on Google Colab: copy files and install required libraries.

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    GIT_ROOT = 'https://github.com/blueprints-for-text-analytics-python/blueprints-text/raw/master'
    os.system(f'wget {GIT_ROOT}/ch03/setup.py')

%run -i setup.py

## Load Python Settings<div class="tocSkip"/>

Common imports, defaults for formatting in Matplotlib, Pandas etc.

In [ ]:
%run "$BASE_DIR/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

# Blueprint: download and interpret robots.txt

In [ ]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://www.reuters.com/robots.txt")
rp.read()

In [ ]:
rp.can_fetch("*", "https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml")

In [ ]:
rp.can_fetch("*", "https://www.reuters.com/finance/stocks/option")

# Blueprint: finding URLs from sitemap.xml

In [ ]:
# might need to install xmltodict
import xmltodict
import requests

sitemap = xmltodict.parse(requests.get('https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml').text)

In [ ]:
# just see some of the URLs
urls = [url["loc"] for url in sitemap["urlset"]["url"]]
print("\n".join(urls[0:3]))

# Blueprint: finding URLs from RSS

Reuters removed its RSS feed after the book was published. We therefore use a saved copy from the Internet archive

In [ ]:
# might need to install feedparser
import feedparser
feed = feedparser.parse('http://web.archive.org/web/20200613003232if_/http://feeds.reuters.com/Reuters/worldNews')

In [ ]:
[(e.title, e.link) for e in feed.entries]

In [ ]:
[e.id for e in feed.entries]

# Example: Downloading HTML pages with Python

In [ ]:
%%time
s = requests.Session()
for url in urls[0:10]:
    # get the part after the last / in URL and use as filename
    file = url.split("/")[-1]
    
    r = s.get(url)
    with open(file, "w+b") as f:
        f.write(r.text.encode('utf-8'))

In [ ]:
with open("urls.txt", "w+b") as f:
    f.write("\n".join(urls).encode('utf-8'))

# Blueprint: Extraction with regular expressions

We first have to download a single article

In [ ]:
url = 'https://www.reuters.com/article/us-health-vaping-marijuana-idUSKBN1WG4KT'

file = url.split("/")[-1] + ".html"

r = requests.get(url)

with open(file, "w+") as f:
    f.write(r.text)

In [ ]:
import re
with open(file, "r") as f:
    html = f.read()
    g = re.search(r'<title>(.*)</title>', html, re.MULTILINE|re.DOTALL)
    if g:
        print(g.groups()[0])

# Using an HTML parser for extraction

Reuters changed its content structure after the book was published. Unfortunately, they *obfuscated* the content so that the methods in the book don't work without massive changes.

In this notebook, we stick to the text in the book and download the articles from the Internet archive which still has the old HTML structure.

In [ ]:
WA_PREFIX = "http://web.archive.org/web/20200118131624/"
html = s.get(WA_PREFIX + url).text

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
soup.select("h1.ArticleHeader_headline")

## Blueprint: extracting the title/headline

In [ ]:
soup.h1

In [ ]:
soup.h1.text

In [ ]:
soup.title.text

In [ ]:
soup.title.text.strip()

## Blueprint: extracting the article text

In [ ]:
soup.select_one("div.StandardArticleBody_body").text

## Blueprint: extracting image captions

In [ ]:
soup.select("div.StandardArticleBody_body figure")

Variant

In [ ]:
soup.select("div.StandardArticleBody_body figure img")

In [ ]:
soup.select("div.StandardArticleBody_body figcaption")

## Blueprint: extracting the URL

In [ ]:
soup.find("link", {'rel': 'canonical'})['href']

In [ ]:
soup.select_one("link[rel=canonical]")['href']

## Blueprint: extracting list information (authors)

In [ ]:
soup.find("meta", {'name': 'Author'})['content']

Variant

In [ ]:
sel = "div.BylineBar_first-container.ArticleHeader_byline-bar div.BylineBar_byline span"
soup.select(sel)

In [ ]:
[a.text for a in soup.select(sel)]

## Blueprint: Extracting text of links (section)


In [ ]:
soup.select_one("div.ArticleHeader_channel a").text

## Blueprint: Extracting reading time

In [ ]:
soup.select_one("p.BylineBar_reading-time").text

## Blueprint: extracting attributes (id)

In [ ]:
soup.select_one("div.StandardArticle_inner-container")['id']

Alternative: URL

## Blueprint: Extracting Attribution

In [ ]:
soup.select_one("p.Attribution_content").text

## Blueprint: Extracting Timestamp

In [ ]:
ptime = soup.find("meta", { 'property': "og:article:published_time"})['content']
print(ptime)

In [ ]:
from dateutil import parser
parser.parse(ptime)

In [ ]:
parser.parse(soup.find("meta", { 'property': "og:article:modified_time"})['content'])

# Blueprint: Spidering

In [ ]:
import requests
from bs4 import BeautifulSoup
import os.path
from dateutil import parser

def download_archive_page(page):
    filename = "page-%06d.html" % page
    if not os.path.isfile(filename):
        url = "https://www.reuters.com/news/archive/" + \
              "?view=page&page=%d&pageSize=10" % page
        r = requests.get(url)
        with open(filename, "w+") as f:
            f.write(r.text)

def parse_archive_page(page_file):
    with open(page_file, "r") as f:
        html = f.read()
    soup = BeautifulSoup(html, 'html.parser')
    hrefs = ["https://www.reuters.com" + a['href'] 
               for a in soup.select("article.story div.story-content a")]
    return hrefs

def download_article(url):
    # check if article already there
    filename = url.split("/")[-1] + ".html"
    if not os.path.isfile(filename):
        r = requests.get(url)
        with open(filename, "w+") as f:
            f.write(r.text)

def parse_article(article_file):
    def find_obfuscated_class(soup, klass):
        return soup.find_all(lambda tag: tag.has_attr("class") and (klass in " ".join(tag["class"])))
    
    with open(article_file, "r") as f:
        html = f.read()
    r = {}
    soup = BeautifulSoup(html, 'html.parser')
    r['url'] = soup.find("link", {'rel': 'canonical'})['href']
    r['id'] = r['url'].split("-")[-1]
    r['headline'] = soup.h1.text
    r['section'] = find_obfuscated_class(soup, "ArticleHeader-channel")[0].text
    
    r['text'] = "\n".join([t.text for t in find_obfuscated_class(soup, "Paragraph-paragraph")])
    r['authors'] = find_obfuscated_class(soup, "Attribution-attribution")[0].text
    r['time'] = soup.find("meta", { 'property': "og:article:published_time"})['content']
    return r

In [ ]:
# download 10 pages of archive
for p in range(1, 10):
    download_archive_page(p)

In [ ]:
# parse archive and add to article_urls
import glob

article_urls = []
for page_file in glob.glob("page-*.html"):
    article_urls += parse_archive_page(page_file)

In [ ]:
# download articles
for url in article_urls:
    download_article(url)

In [ ]:
# arrange in pandas DataFrame
import pandas as pd

df = pd.DataFrame()
for article_file in glob.glob("*-id???????????.html"):
    df = df.append(parse_article(article_file), ignore_index=True)
df['time'] = pd.to_datetime(df.time)

In [ ]:
df

In [ ]:
df.sort_values("time")

In [ ]:
%matplotlib inline
df[df["time"]>='2020-01-01'].set_index("time").resample('D').agg({'id': 'count'}).plot.bar()

# Blueprint Density extraction

In [ ]:
from readability import Document

doc = Document(html)
doc.title()

In [ ]:
doc.short_title()

In [ ]:
doc.summary()

In [ ]:
doc.url

In [ ]:
density_soup = BeautifulSoup(doc.summary(), 'html.parser')
density_soup.body.text

# Blueprint: Scrapy

Unfortunately, the code for `scrapy` cannot be changed easily. One more argument for using *up to date* separate libraries. In this version, it still collects the titles of the articles but not more.

In [ ]:
# might need to install scrapy
import scrapy
import logging


class ReutersArchiveSpider(scrapy.Spider):
    name = 'reuters-archive'
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'FEED_FORMAT': 'json',
        'FEED_URI': 'reuters-archive.json'
    }
    
    start_urls = [
        'https://www.reuters.com/news/archive/',
    ]

    def parse(self, response):
        for article in response.css("article.story div.story-content a"):
            yield response.follow(article.css("a::attr(href)").extract_first(), self.parse_article)

        next_page_url = response.css('a.control-nav-next::attr(href)').extract_first()
        if (next_page_url is not None) & ('page=2' not in next_page_url):
            yield response.follow(next_page_url, self.parse)

    def parse_article(self, response):
        yield {
          'title': response.css('h1::text').extract_first().strip(),
        }

In [ ]:
# this can be run only once from a Jupyter notebook due to Twisted
from scrapy.crawler import CrawlerProcess
process = CrawlerProcess()

process.crawl(ReutersArchiveSpider)
process.start()

In [ ]:
glob.glob("*.json")

In [ ]:
!cat 'reuters-archive.json'